### Retrieve details from widget

In [0]:
dbutils.widgets.text("account_id", "")
dbutils.widgets.text("client_id", "")
dbutils.widgets.text("client_secret", "")
dbutils.widgets.text("sp_id", "")

In [0]:
account_id = dbutils.widgets.get("account_id")
client_id = dbutils.widgets.get("client_id")
client_secret = dbutils.widgets.get("client_secret")
spi_id = dbutils.widgets.get("sp_id")

## Check Service Principal Permissions at Account level

In [0]:

from databricks.sdk import AccountClient

a = AccountClient(
    host="https://accounts.cloud.databricks.com/",
    account_id= account_id,
    client_id= client_id,
    client_secret= client_secret
)

acc_sp = a.service_principals.get(id=spi_id)

In [0]:
roles = [r.value for r in acc_sp.roles]

isAccountAdmin = False
for role in roles:
    if role == "account_admin":
        isAccountAdmin = True
        break

# print(f"SP Is Admin: {isAccountAdmin}")
if isAccountAdmin:
    displayHTML("<h3 style='color:green;'>Validation Successful: SP is an Account Admin</h3>")
else:
    displayHTML("<h3 style='color:red;'>Validation Failed: SP is not an Account Admin</h3>")

## Check Service Principal Permissions at Workspace level

In [0]:
from databricks.sdk import WorkspaceClient

wc = WorkspaceClient()

ws_sp = wc.service_principals.get(id=spi_id)

In [0]:
groups = [g.display for g in ws_sp.groups]

isWorkspaceAdmin = False
for group in groups:
    if group == "admins":
        isWorkspaceAdmin = True
        break

# displayHTML(f"SP Is Workspace Admin: {isWorkspaceAdmin}")
if isWorkspaceAdmin:
    displayHTML("<h3 style='color:green;'>Validation Successful: SP is a Workspace Admin</h3>")
else:
    displayHTML("<h3 style='color:red;'>Validation Failed: SP is not a Workspace Admin</h3>")

## Check if Dashboard Exists

In [0]:
from databricks.sdk import WorkspaceClient

# Initialize the Workspace client (uses notebook default auth)
w = WorkspaceClient()

# Widget to input the dashboard name you want to check
# dbutils.widgets.text("dashboard_name", "", "Dashboard Name")
# dashboard_name = dbutils.widgets.get("dashboard_name")
dashboard_name = "[SAT] Security Analysis Tool - Assessment Results"

# Build and send the list request with a search term
dashboards = w.lakeview.list()

# Iterate and check for exact name match
found = False
for dash in dashboards:
    if dash.display_name == dashboard_name:
        displayHTML(f"<h3 style='color:green;'> Validation Successful: Found dashboard '{dashboard_name}'</h3>")
        found = True
        break

if not found:
    displayHTML("<h3 style='color:red;'>Validation Failed: Dashboard does not exist</h3>")

## Send Confirmation [TBD]